In [25]:
import h5py
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
from sklearn.model_selection import train_test_split
import timm

In [5]:
electron_data = "/notebooks/gsoc-prep-23/ml4sci/data/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5"
photon_data = "/notebooks/gsoc-prep-23/ml4sci/data/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5.1"

In [6]:
# Read data; Sanity check
with h5py.File(electron_data, 'r') as file:
    print(file.keys())
    X = file['X'][:]
    y = file['y'][:]

with h5py.File(photon_data, 'r') as file:
    print(file.keys())
    X = np.concatenate((X,file['X'][:]))
    y = np.concatenate((y,np.zeros_like(y, dtype = 'float32', )))

<KeysViewHDF5 ['X', 'y']>
<KeysViewHDF5 ['X', 'y']>


In [7]:
# Split the array into subsets based on class label
X_0 = X[y == 0, :]
X_1 = X[y == 1, :]

# Get the number of rows in each subset
n_rows_0 = X_0.shape[0]
n_rows_1 = X_1.shape[0]

# Sample half of the rows from each subset
idx_0 = np.random.choice(n_rows_0, size=int(n_rows_0/2), replace=False)
idx_1 = np.random.choice(n_rows_1, size=int(n_rows_1/2), replace=False)

# Extract the corresponding rows from each subset and concatenate them
X_sample = np.concatenate((X_0[idx_0, :], X_1[idx_1, :]), axis=0)
y_sample = np.concatenate((np.zeros(len(idx_0)), np.ones(len(idx_1))), axis=0)

In [8]:
# Split into training and validation set
X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.25, random_state=42, shuffle=True)

In [9]:
batch_size = 4
# Create a tf.data.Dataset from the numpy arrays for training and validation
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
val_dataset = val_dataset.batch(batch_size)

2023-03-12 05:30:02.180302: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1529856000 exceeds 10% of free system memory.
2023-03-12 05:30:04.003503: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1529856000 exceeds 10% of free system memory.


In [24]:
model_1 = tf.keras.applications.efficientnet.EfficientNetB0(
        include_top=False,
    weights='imagenet',
    input_shape=(32,32,2),
)



ValueError: The input must have 3 channels; Received `input_shape=(32, 32, 2)`

In [20]:
num_classes = 1

model_head = tf.keras.Sequential([
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(num_classes, activation='sigmoid')
])

model = tf.keras.Model(inputs=model_1.inputs, outputs=model_head(model.outputs))

AttributeError: 'EfficientNet' object has no attribute 'inputs'

In [16]:
# Compile the model
model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

AttributeError: 'EfficientNet' object has no attribute 'compile'

In [12]:
# Train the model
model_1.fit(train_dataset,validation_data=val_dataset, epochs=10, batch_size=8)

2023-03-12 05:30:06.254475: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1529856000 exceeds 10% of free system memory.


Epoch 1/10
18852/46688 [===========>..................] - ETA: 3:00 - loss: 0.7061 - accuracy: 0.5008

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import roc_auc_score

# Make predictions on the test data
y_pred = model_1.predict(X_test)

# Calculate ROC AUC score
roc_auc = roc_auc_score(y_test, y_pred)

# Print the ROC AUC score
print('ROC AUC score:', roc_auc)
